# Assignment 1: Labor Supply and Children

## 0: Setup and solve model

In [15]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from DynLaborFertModel import DynLaborFertModelClass

# save and load models
save = False
load = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
model = DynLaborFertModelClass(name='baseline', load=load)

if model.sol.solved == False:
    model.solve(do_print=True)

model.simulate()
print(model)

if save:
    model.save()

Modelclass: DynLaborFertModelClass
Name: baseline

namespaces: ['par', 'sim', 'sol']
other_attrs: []
savefolder: saved
cpp_filename: None

par:
 T = 10 [int]
 rho = 0.9803921568627451 [float]
 beta_0 = 0.1 [float]
 beta_1 = 0.05 [float]
 eta = -2.0 [float]
 gamma = 2.5 [float]
 alpha = 0.3 [float]
 w = 1.0 [float]
 tau = 0.1 [float]
 p_birth = 0.1 [float]
 c_cost = 0 [int]
 y0 = 0.0 [float]
 y1 = 0.0 [float]
 spousegrid = ndarray with shape = (2,) [dtype: int32]
 p_spouse = 1 [int]
 r = 0.02 [float]
 a_max = 5.0 [float]
 a_min = -10.0 [float]
 Na = 50 [int]
 k_max = 20.0 [float]
 Nk = 20 [int]
 Nn = 2 [int]
 simT = 10 [int]
 simN = 1000 [int]
 target_drop_y0 = -0.1 [float]
 a_grid = ndarray with shape = (50,) [dtype: float64]
 k_grid = ndarray with shape = (20,) [dtype: float64]
 n_grid = ndarray with shape = (2,) [dtype: int32]
 w_vec = ndarray with shape = (10,) [dtype: float64]
 memory, gb: 0.0

sim:
 c = ndarray with shape = (1000, 10) [dtype: float64]
 h = ndarray with shape = (10

## 1. Estimate $\beta_1$

In [17]:
model_est = DynLaborFertModelClass(name='estimation', load=load, par={'Nk':20, 'Na':50})
kwargs = {'method':'brentq', 'bracket':(0.04, 0.06), 'maxiter': 10, 'xtol':0.001}

if model_est.sol.solved == False:
    model_est.estimate(**kwargs)

if save:
    model_est.save()

## 2. Marshall elasticity

In [18]:
model_tax = DynLaborFertModelClass(name='tax', load=load)
model_tax.par.tau = model.par.tau*1.01

if model_tax.sol.solved == False:
    model_tax.solve()

model_tax.simulate()

if save:
    model_tax.save()

In [19]:
marshall_vec = np.empty(model.par.simT)
model_sim = model.copy() #extra model for performing simulations

for t in range(model.par.simT):
    model_sim.simulate() 
    model_sim.simulate_tau(tau=t, shock_model=model_tax)
    marshall = ((model_sim.sim.h[:,t]/model.sim.h[:,t]-1)*100).mean() # response in shock period
    marshall_vec[t]=marshall

## 3.  Introduce spouse

In [20]:
model_spouse = DynLaborFertModelClass(name='spouse', load=load)
model_spouse.par.y0 = 0.1
model_spouse.par.y1 = 0.01

if model_spouse.sol.solved == False:
    model_spouse.solve()

if save:
    model_spouse.save()

## 4. Childcare cost

In [21]:
model_childcare = DynLaborFertModelClass(name='childcare_cost', load=load)
model_childcare.par.c_cost = 0.05

if model_childcare.sol.solved == False:
    model_childcare.solve()
    
model_childcare.simulate()

if save:
    model_childcare.save()

## 6. Stochastic spouse

In [22]:
model_stspouse = DynLaborFertModelClass(name='st_spouse', load=load)
model_stspouse.par.p_spouse = 0.8

if model_stspouse.sol.solved == False:
    model_stspouse.solve()

model_stspouse.simulate()

if save:
    model_stspouse.save()